In [21]:
import sys
from pathlib import Path

root_dir = Path().absolute()
# Strip subdirectory if notebook started inside project subfolders
if root_dir.parts[-1:] == ('airquality',):
    root_dir = Path(*root_dir.parts[:-1])
if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])
if root_dir.parts[-1:] == ('local_notebooks',):
    root_dir = Path(*root_dir.parts[:-1])
if root_dir.parts[-1:] == ('collab_notebooks',):
    root_dir = Path(*root_dir.parts[:-1])
root_dir = str(root_dir) 
print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from airquality.config import HopsworksSettings
settings = HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/oscarreinagustafsson/Desktop/Projects/Master/machine-learning/ID2223-scalable-machine-learning
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [22]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from airquality import util
from airquality import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [23]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-18 13:43:43,192 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 13:43:43,194 INFO: Initializing external client
2025-11-18 13:43:43,195 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 13:43:45,046 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279132


'{"country": "sweden", "city": "skelleftea-kv-pantern", "street": "viktoriagatan", "aqicn_url": "https://api.waqi.info/feed/@13982/", "latitude": "64.75", "longitude": "20.96"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [24]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [25]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,7.0,sweden,skelleftea-kv-pantern,viktoriagatan,2025-11-18,https://api.waqi.info/feed/@13982/


In [26]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [27]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 64.75°N 21.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,-7.40,0.0,7.491114,324.782318,skelleftea-kv-pantern
1,2025-11-19,-8.80,0.0,8.287822,267.510498,skelleftea-kv-pantern
2,2025-11-20,-10.70,0.0,7.895416,316.847595,skelleftea-kv-pantern
3,2025-11-21,-9.80,0.0,9.957108,319.398773,skelleftea-kv-pantern
4,2025-11-22,-12.85,0.0,7.636753,315.000092,skelleftea-kv-pantern
5,2025-11-23,-13.85,0.0,2.545584,261.869995,skelleftea-kv-pantern
6,2025-11-24,-4.15,0.6,9.826088,8.426887,skelleftea-kv-pantern


In [28]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 356.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [29]:
start_date = today - datetime.timedelta(days=4)

last_3_days_aq_query = (
    air_quality_fg.select(["city", "pm25", "date"]).filter((air_quality_fg.date >= str(start_date)) & (air_quality_fg.date <= str(today)))
)

aq_3_days_history_df = last_3_days_aq_query.read()

###############
# Get today's data
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)

for lagged_day in [1,2,3]:
    lag_date = today - pd.Timedelta(days=lagged_day)
    mask = (aq_3_days_history_df['city'] == city) \
        & (aq_3_days_history_df['date'].dt.date == lag_date)
    lag_value = aq_3_days_history_df.loc[mask, 'pm25']
    if not lag_value.empty:
        aq_today_df[f'lagged_{lagged_day}'] = lag_value.values[0]
    else:
        aq_today_df[f'lagged_{lagged_day}'] = None

hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 
Coordinates 64.75°N 21.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [30]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 13:43:53,536 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279132/fs/1265741/fg/1596030


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279132/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-18 13:44:10,728 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 13:44:40,331 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 13:46:49,692 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 13:46:49,894 INFO: Waiting for log aggregation to finish.
2025-11-18 13:47:08,847 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 733242
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T12:43:53.000536Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store


# WAIT, there's more...

## A-grade: make pipeline for all sensors
We need to make sure to execute the above cells for each of the sensors that we can find in the Skelleftea area 

In [31]:
try:
    extra_sensors = f"{root_dir}/data/extra_sensors/"
    util.check_data_folder_structure(root_dir)

    project = hopsworks.login()
    fs = project.get_feature_store() 
    secrets = hopsworks.get_secrets_api()

    import json

    try:
        with open(f"{root_dir}/data/extra_sensor_settings.json", 'r') as file:
            settings_json = json.load(file)["extra_sensors"]
    except FileNotFoundError:
        print("Settings file for extra sensors not found.")
        sys.exit(1)

    for csv_file_name in os.listdir(extra_sensors):
        print(f"Processing file: {csv_file_name}")
        fs = project.get_feature_store() 
        secrets = hopsworks.get_secrets_api()

        # This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
        AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
        location_str = secrets.get_secret("SENSOR_LOCATION_JSON_" + csv_file_name).value
        location = json.loads(location_str)

        country=location['country']
        city=location['city']
        street=location['street']
        aqicn_url=location['aqicn_url']
        latitude=location['latitude']
        longitude=location['longitude']

        today = datetime.date.today()

        import re

        ####################
        # Retrieve feature groups
        # split the .csv extension from the csv_file_name: not compatible with Hopsworks feature group names
        csv_file_name_no_ext = os.path.splitext(csv_file_name)[0]
        # replace - with _
        csv_file_name_no_ext = re.sub(r'[-\s]', '_', csv_file_name_no_ext)

        air_quality_fg = fs.get_feature_group(
            name=f'air_quality_{csv_file_name_no_ext}',
            version=2,
        )
        weather_fg = fs.get_feature_group(
            name=f'weather_{csv_file_name_no_ext}',
            version=1,
        )
        ###############################
        
        start_date = today - datetime.timedelta(days=4)

        last_3_days_aq_query = (
            air_quality_fg.select(["city", "pm25", "date"]).filter((air_quality_fg.date >= str(start_date)) & (air_quality_fg.date <= str(today)))
        )

        aq_3_days_history_df = last_3_days_aq_query.read()

        ###############
        # Get today's data
        aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)

        for lagged_day in [1,2,3]:
            lag_date = today - pd.Timedelta(days=lagged_day)
            mask = (aq_3_days_history_df['city'] == city) \
                & (aq_3_days_history_df['date'].dt.date == lag_date)
            lag_value = aq_3_days_history_df.loc[mask, 'pm25']
            if not lag_value.empty:
                aq_today_df[f'lagged_{lagged_day}'] = lag_value.values[0]
            else:
                aq_today_df[f'lagged_{lagged_day}'] = None

        hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
        hourly_df = hourly_df.set_index('date')

        # We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
        # We only want the daily weather data, so only get weather at 12:00
        daily_df = hourly_df.between_time('11:59', '12:01')
        daily_df = daily_df.reset_index()
        daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
        daily_df['date'] = pd.to_datetime(daily_df['date'])
        daily_df['city'] = city

        air_quality_fg.insert(aq_today_df)
        weather_fg.insert(daily_df, wait=True)

except Exception as e:
    print(f"An error occurred: {e}")
    sys.exit(1)

Checking data folder structure at path: /Users/oscarreinagustafsson/Desktop/Projects/Master/machine-learning/ID2223-scalable-machine-learning/data/extra_sensors
Error: Missing files in data directory: ostermalmsgatan-vasterbotten.csv, vittervagen-vasterbotten.csv
2025-11-18 13:47:08,869 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 13:47:08,873 INFO: Initializing external client
2025-11-18 13:47:08,873 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 13:47:10,546 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279132
Processing file: ac846-skelleftea.csv
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.30s) 
Coordinates 65.0°N 20.0°E
Elevation 202.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
2025-11-18 13:47:17,904 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279132/fs/1265741/fg/1730946


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_ac846_skelleftea_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279132/jobs/named/air_quality_ac846_skelleftea_2_offline_fg_materialization/executions
2025-11-18 13:47:31,475 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279132/fs/1265741/fg/1719647


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_ac846_skelleftea_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279132/jobs/named/weather_ac846_skelleftea_1_offline_fg_materialization/executions
2025-11-18 13:47:49,088 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 13:47:55,629 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 13:49:55,385 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 13:49:55,559 INFO: Waiting for log aggregation to finish.
2025-11-18 13:50:14,508 INFO: Execution finished successfully.
Processing file: mobackavagen-skelleftea.csv
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.05s) 
Coordinates 64.75°N 21.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
2025-11-18 13:50:20,150 INFO: 	1 expectation(s) i

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_mobackavagen_skelleftea_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279132/jobs/named/air_quality_mobackavagen_skelleftea_2_offline_fg_materialization/executions
2025-11-18 13:50:34,883 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279132/fs/1265741/fg/1718661


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_mobackavagen_skelleftea_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279132/jobs/named/weather_mobackavagen_skelleftea_1_offline_fg_materialization/executions
2025-11-18 13:50:52,030 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-18 13:50:55,314 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 13:53:08,061 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-18 13:53:14,531 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-11-18 13:53:14,963 INFO: Waiting for log aggregation to finish.
2025-11-18 13:53:14,963 INFO: Execution finished successfully.
